In [13]:
import os
import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm, colors
import seaborn as sns

from scipy.stats import binned_statistic_2d

%config InlineBackend.figure_format = 'retina'
#CMAP = cm.get_cmap('RdBu')
CMAP = cm.get_cmap('twilight')
COLOUR_PALETTE = sns.color_palette('tab10')[1:]
COLOUR = 'dodgerblue'
BIN_COLOUR = 'tomato'
REF_RESULTS = pd.read_csv('reference_results.csv')

## Auxiliar Functions

In [14]:
def bin_values(bins, x_values, y_values, y_err):
    """Values are binned using the given bins.
    
    Parameters
    ==========
    bins: array
        array of bins
    x_values: array
        array of x-axis values
    y_values: array
        array of y-axis values
    y_err: array
        array of y-axis errors
        
    Returns
    =======
    delta_bins, std_bins, err_bins: arrays
        Arrays of
    """
    
    digitized = np.digitize(x_values, bins)
    delta_bins, std_bins, err_bins = [], [], []
    
    for i in range(1, len(bins)):
        if len(y_values[digitized == i])>0:
            y = y_values[digitized == i]
            w = 1/y_err[digitized == i]**2
            
            wmean = np.average(y, weights=w)
            werr = np.sqrt(np.sum(w*(y-wmean)**2)/((len(w)-1)*np.sum(w)))
            wstd = np.sqrt(np.average((y-wmean)**2, weights=w))
            
            delta_bins.append(wmean)
            std_bins.append(wstd)
            err_bins.append(werr)
        else:
            delta_bins.append(np.nan)
            std_bins.append(np.nan)
            err_bins.append(np.nan)

    delta_bins = np.array(delta_bins) 
    std_bins = np.array(std_bins)
    err_bins = np.array(err_bins)
    
    return delta_bins, std_bins, err_bins

def flatten_array(array):
    """Flattens an array/list with multiple 
    sub-arrays/lists with different sizes.
    
    Parameters
    ==========
    array: list-like
        List/array to flatten.
        
    Returns
    =======
    flat_array: list-like
        Flatten array.
    """
    flat_array = np.array([item for sublist in array 
                                  for item in sublist])
    
    return flat_array

def get_extent(ret):
    """Extracts the extent from the results of
    `scipy.stats.binned_statistic_2d()` to plot
    2D binned data.
    
    Parameters
    ==========
    ret: `scipy.stats.binned_statistic_2d()` results
    
    Returns
    =======
    extent: extent to plot 2D binned data.
    """
    extent = [ret[1][0], ret[1][-1], ret[2][0], ret[2][-1]]
    
    return extent

def extract_parameters(sn_name, param, ref_bands, 
                                       nir_bands, n_epochs):
    """Extracts the parameters of a SN (e.g., Jmax and 
    metrics) for the diagnosis plots.
    
    Parameters
    ==========
    sn_name: str
        SN name (e.g., SN2004eo).
    params: str
        SN parameter (e.g., Jmax, Tmax, st).
    ref_bands: str
        Bands used for the reference fits. E.g., `AllBands`,
        `Allminusu`, 'grJH', 'gr' and 'gp' (gaussian process).
    nir_bands: str
        NIR bands used for the simulations fits. E.g., `J`, 
        `H` or `JH`.
    n_epochs: int
        Number of epochs for the combinatories of the 
        simulations.
        
    Returns
    =======
    residuals: array
        Parameter residuals between the reference fits and 
        simulations fits.
    yerr: array
        Uncertainty of the parameter.
    m1, m2, m3: arrays
        Metrics used.
    """
    
    ref_bands_options = ['AllBands', 'Allminusu', 
                         'grJH', 'gr', 'gp']
    ref_message = ('only one of the followingis a valid ' 
                  f'option for `red_bands`: {ref_bands_options}')
    assert ref_bands in ref_bands_options, ref_message
    
    nir_bands_options = ['J', 'H', 'JH']
    nir_message = ('only one of the followingis a valid ' 
                  f'option for `nir_bands`: {nir_bands_options}')
    assert nir_bands in nir_bands_options, nir_message
    
    # ref
    ref_df = REF_RESULTS[REF_RESULTS.name==sn_name]
    y_ref = ref_df[f'{param}_{ref_bands}'].values[0]

    # sim
    sim_file = os.path.join('sim_fits', 
                            sn_name, 
                            (f'{sn_name}_{n_epochs}'
                             f'epochs_results_{nir_bands}.csv'))
    sim_df = pd.read_csv(sim_file)
    y = sim_df[param].values
    yerr = sim_df[param+'_err'].values     

    residuals = y-y_ref
    
    m1 = sim_df.m1.values
    m2 = sim_df.m2.values
    m3 = sim_df.m3.values

    return residuals, yerr, m1, m2, m3

## Diagnosis Plots functions

In [74]:
def plot_m1(param, ref_bands, nir_bands, step=4, mask_epochs=None):
    """Diagnosis plots with for simulations with 3 epochs.
    
    Parameters
    ==========
    sn_name: str
        SN name (e.g., SN2004eo).
    params: str
        SN parameter (e.g., Jmax, Tmax, st).
    ref_bands: str
        Bands used for the reference fits. E.g., `AllBands`,
        `Allminusu`, 'grJH', 'gr' and 'gp' (gaussian process).
    nir_bands: str
        NIR bands used for the simulations fits. E.g., `J`, 
        `H` or `JH`.
    step: int, default `4`
        Step size in metric 1 to use with binned data.
    mask_epochs: list-like or tuple, default `None`
        Range in days where to mask metric 1 to calculate
        the global offset and scatter. E.g., (-8, 10).
    """
    # colour for outliers
    colour_palette = COLOUR_PALETTE.copy()
    counter = (i for i in range(len(colour_palette)))
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    params_dict = {'residuals':[], 'yerr':[], 'm1':[]}
    for sn_name in REF_RESULTS.name:
        try:
            parameters = extract_parameters(sn_name, param, 
                                            ref_bands, nir_bands, 1)
            residuals, yerr, m1, _, _ = parameters
            
            params_dict['m1'].append(m1)
            params_dict['yerr'].append(yerr)
            params_dict['residuals'].append(residuals)
            
            #detect outliers
            if np.any(np.abs(residuals)>0.5):
                color = colour_palette[next(counter)]
                label = sn_name
            else:
                color = COLOUR
                label = None
            
            # plot
            ax.errorbar(m1, residuals, yerr=yerr, 
                        fmt='o', color=color, alpha=0.4,
                        mec=None, label=label)
        except:
            pass

    # fatten arrays
    for key, values in params_dict.items():
            params_dict[key] = flatten_array(values)

    # plot bins
    min_val = np.round(params_dict['m1'], 0).min()
    max_val = np.round(params_dict['m1'], 0).max()
    bins = np.arange(min_val, max_val+step, step)
    delta_bins, std_bins, err_bins = bin_values(bins, 
                                                params_dict['m1'], 
                                                params_dict['residuals'], 
                                                params_dict['yerr'])
    
    ax.errorbar(bins[:-1], delta_bins, err_bins, 
                color=BIN_COLOUR, ms=8, mec='k', fmt='h', zorder=30, 
                ecolor=BIN_COLOUR, capsize=8, capthick=2, elinewidth=2.5)
    
    residuals, yerr = params_dict['residuals'], params_dict['yerr']
    m1 = params_dict['m1']
    if mask_epochs is not None:
        xmin, xmax = mask_epochs
    else:
        xmin, xmax = m1.min(), m1.max()
        
    mask = (m1>=xmin) & (m1<=xmax)
    y, w = residuals[mask], 1/yerr[mask]**2   
    
    wmean = np.average(y, weights=w)
    werr = np.sqrt(np.sum(w*(y- wmean)**2)/((len(w)-1)*np.sum(w)))
    wstd = np.sqrt(np.average((y - wmean)**2, weights=w))
    
    ax.hlines(wmean, xmin, xmax, colors=BIN_COLOUR, 
                          linestyles='dashed', lw=3, zorder=20)
    ax.hlines(wmean+wstd, xmin, xmax, colors=BIN_COLOUR, 
                          linestyles='dotted', lw=3, zorder=20)
    ax.hlines(wmean-wstd, xmin, xmax, colors=BIN_COLOUR, 
                          linestyles='dotted', lw=3, zorder=20)
    
    if 'max' in param:
        param = param[0]+'$_{max}$'
        
    m1_label = 'Phase of the closest\nNIR epoch to T$_{max}$'
    title = f'Reference: {ref_bands} ($\Delta=${wmean:.3f}, $\sigma=${wstd:.3f})'
    residual_label = f'$\Delta${param} (mag)\n(Sim$-$Ref)'
        
    ax.axhline(0, color='k', ls='--', zorder=20)
    ax.set_xlabel(m1_label.replace('\n', ''), fontsize=16)
    ax.set_ylabel(residual_label.replace('\n', ''), fontsize=16)
    ax.set_title(title, fontsize=16)
    ax.tick_params(labelsize=16)
    ax.set_ylim(-0.15, 0.15)
    if next(counter)!=0:
        ax.legend(fontsize=12, ncol=2)
        
    fig.tight_layout()
    plt.show()
    
def plot_m2(param, ref_bands, nir_bands, binned=True, 
                                    step1=4, step2=10, mask_epochs=None):
    """Diagnosis plots with for simulations with 2 epochs.
    
    Parameters
    ==========
    sn_name: str
        SN name (e.g., SN2004eo).
    params: str
        SN parameter (e.g., Jmax, Tmax, st).
    ref_bands: str
        Bands used for the reference fits. E.g., `AllBands`,
        `Allminusu`, 'grJH', 'gr' and 'gp' (gaussian process).
    nir_bands: str
        NIR bands used for the simulations fits. E.g., `J`, 
        `H` or `JH`.
    binned: bool, default `True`
        If `True`, plots the 2D binned version.
    step1: int, default `4`
        Step size in metric 1 to use with binned data.
    step2: int, default `10`
        Step size in metric 2 to use with binned data.
    mask_epochs: list-like or tuple, default `None`
        Range in days where to mask metric 1 to calculate
        the global offset and scatter. E.g., (-8, 10).
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    fig2, ax2 = plt.subplots(2, figsize=(8, 8))
    
    # colour bar
    norm = colors.Normalize(vmin=-0.11, vmax=0.11)
    s_m = plt.cm.ScalarMappable(cmap=CMAP, norm=norm)
    s_m.set_array([])
    
    params_dict = {'residuals':[], 'yerr':[], 'm1':[], 'm2':[]}
    for sn_name in REF_RESULTS.name:
        try:
            parameters = extract_parameters(sn_name, param, 
                                            ref_bands, nir_bands, 2)
            residuals, yerr, m1, m2, _ = parameters

            params_dict['m1'].append(m1)
            params_dict['m2'].append(m2)
            params_dict['residuals'].append(residuals)
            params_dict['yerr'].append(yerr)
            
            s, marker, alpha = 50, 'o', 0.4
            if not binned:
                ax.scatter(m1, m2, s=s, marker=marker, 
                             edgecolors='k', alpha=alpha,
                             c=s_m.to_rgba(residual))
                
            # separated plots
            ax2[0].errorbar(m1, residuals, yerr=yerr, 
                            fmt=marker, color=COLOUR, alpha=alpha,
                            mec=None)
            ax2[1].errorbar(m2, residuals, yerr=yerr, 
                            fmt=marker, color=COLOUR, alpha=alpha,
                            mec=None)
        except:
            pass
        
    if binned:
        # flatten arrays
        for key, values in params_dict.items():
            params_dict[key] = flatten_array(values)
        
        ret = binned_statistic_2d(params_dict['m1'], params_dict['m2'], 
                                  params_dict['residuals'], 
                                  statistic='mean', bins = [20, 40])

        extent = get_extent(ret)
        ax.imshow(ret[0].T, cmap=CMAP, extent=extent, norm=norm,
                  interpolation='nearest', origin='lower', aspect='auto')
        
    # separate plots
    # bins
    min_val1 = np.round(params_dict['m1'], 0).min()
    max_val1 = np.round(params_dict['m1'], 0).max()
    bins1 = np.arange(min_val1, max_val1+step1, step1)
    delta_bins1, std_bins1, err_bins1 = bin_values(bins1, 
                                                params_dict['m1'], 
                                                params_dict['residuals'], 
                                                params_dict['yerr'])
    
    ax2[0].errorbar(bins1[:-1], delta_bins1, err_bins1, 
                color=BIN_COLOUR, zorder=30, ms=8, mec='k', fmt='h', 
                ecolor=BIN_COLOUR, capsize=8, capthick=2, elinewidth=2.5)
    
    min_val2 = np.round(params_dict['m2'], 0).min()
    max_val2 = np.round(params_dict['m2'], 0).max()
    bins2 = np.arange(min_val2, max_val2+step2, step2)
    delta_bins2, std_bins2, err_bins2 = bin_values(bins2, 
                                                params_dict['m2'], 
                                                params_dict['residuals'], 
                                                params_dict['yerr'])
    
    ax2[1].errorbar(bins2[:-1], delta_bins2, err_bins2, 
                color=BIN_COLOUR, zorder=30, ms=8, mec='k', fmt='h', 
                ecolor=BIN_COLOUR, capsize=8, capthick=2, elinewidth=2.5)
    
    residuals, yerr = params_dict['residuals'], params_dict['yerr']
    m1 = params_dict['m1']
    if mask_epochs is not None:
        xmin, xmax = mask_epochs
    else:
        xmin, xmax = m1.min(), m1.max()
        
    mask = (m1>=xmin) & (m1<=xmax)
    y, w = residuals[mask], 1/yerr[mask]**2   
    
    wmean = np.average(y, weights=w)
    werr = np.sqrt(np.sum(w*(y- wmean)**2)/((len(w)-1)*np.sum(w)))
    wstd = np.sqrt(np.average((y - wmean)**2, weights=w))
    
    ax2[0].hlines(wmean, xmin, xmax, colors=BIN_COLOUR, 
                                  linestyles='dashed', lw=3, zorder=20)
    ax2[0].hlines(wmean+wstd, xmin, xmax, colors=BIN_COLOUR, 
                                  linestyles='dotted', lw=3, zorder=20)
    ax2[0].hlines(wmean-wstd, xmin, xmax, colors=BIN_COLOUR, 
                                  linestyles='dotted', lw=3, zorder=20)
    
    if 'max' in param:
        param = param[0]+'$_{max}$'

    m1_label = 'Phase of the closest\nNIR epoch to T$_{max}$'
    m2_label = 'Mean phase of the\nNIR epochs (days)'
    title = f'Reference: {ref_bands} ($\Delta=${wmean:.3f}, $\sigma=${wstd:.3f})'
    residual_label = f'$\Delta${param} (mag)\n(Sim$-$Ref)'
    
    ax.set_xlabel(m1_label, fontsize=16)
    ax.set_ylabel(m2_label, fontsize=16)
    ax.set_title(title, fontsize=16)
    ax.tick_params(labelsize=16)
    
    # finish colour bar
    cbar = plt.colorbar(s_m, ax=ax)
    cbar.set_label(residual_label, rotation=270, fontsize=16)
    cbar.ax.get_yaxis().labelpad = 25
    cbar.ax.tick_params(labelsize=16)
    
    ax2[0].axhline(0, color='k', ls='--', zorder=20)
    ax2[0].set_xlabel(m1_label.replace('\n', ''), fontsize=16)
    ax2[0].set_ylabel(residual_label, fontsize=16)
    ax2[0].set_title(title, fontsize=16)
    ax2[0].tick_params(labelsize=16)
    ax2[0].set_ylim(-0.15, 0.15)
    
    ax2[1].axhline(0, color='k', ls='--', zorder=20)
    ax2[1].set_xlabel(m2_label.replace('\n', ''), fontsize=16)
    ax2[1].set_ylabel(residual_label, fontsize=16)
    ax2[1].tick_params(labelsize=16)
    ax2[1].set_ylim(-0.15, 0.15)
    
    fig.tight_layout()
    fig2.tight_layout()
    plt.show()
    
def plot_m3(param, ref_bands, nir_bands, binned=True, 
                    step1=4, step2=10, step3=10, mask_epochs=None):
    """Diagnosis plots with for simulations with 3 epochs.
    
    Parameters
    ==========
    sn_name: str
        SN name (e.g., SN2004eo).
    params: str
        SN parameter (e.g., Jmax, Tmax, st).
    ref_bands: str
        Bands used for the reference fits. E.g., `AllBands`,
        `Allminusu`, 'grJH', 'gr' and 'gp' (gaussian process).
    nir_bands: str
        NIR bands used for the simulations fits. E.g., `J`, 
        `H` or `JH`.
    binned: bool, default `True`
        If `True`, plots the 2D binned version.
    step1: int, default `4`
        Step size in metric 1 to use with binned data.
    step2: int, default `10`
        Step size in metric 2 to use with binned data.
    step3: int, default `10`
        Step size in metric 3 to use with binned data.
    mask_epochs: list-like or tuple, default `None`
        Range in days where to mask metric 1 to calculate
        the global offset and scatter. E.g., (-8, 10).
    """
    fig, ax = plt.subplots(2, 2, figsize=(8, 8), 
                           gridspec_kw={'hspace':0, 
                                        'wspace':0})
    fig2, ax2 = plt.subplots(3, figsize=(8, 12))
    
    # colour bar
    norm = colors.Normalize(vmin=-0.11, vmax=0.11)
    s_m = plt.cm.ScalarMappable(cmap=CMAP, norm=norm)
    s_m.set_array([])
    
    params_dict = {'residuals':[], 'yerr':[], 
                   'm1':[], 'm2':[], 'm3':[]}
    for sn_name in REF_RESULTS.name:
        try:
            parameters = extract_parameters(sn_name, param, 
                                            ref_bands, nir_bands, 3)
            residuals, yerr, m1, m2, m3 = parameters

            params_dict['m1'].append(m1)
            params_dict['m2'].append(m2)
            params_dict['m3'].append(m3)
            params_dict['residuals'].append(residuals)
            params_dict['yerr'].append(yerr)
            
            s, marker, alpha = 50, 'o', 0.4
            if not binned:
                ax[0, 0].scatter(m1, m2, s=s, marker=marker, 
                                 edgecolors='k', alpha=alpha,
                                 c=s_m.to_rgba(residual))
                ax[1, 0].scatter(m1, m3, s=s, marker=marker,
                                 edgecolors='k', alpha=alpha,
                                 c=s_m.to_rgba(residual))
                ax[1, 1].scatter(m2, m3, s=s, marker=marker, 
                                 edgecolors='k', alpha=alpha,
                                 c=s_m.to_rgba(residual))
                
            # separate plots
            ax2[0].errorbar(m1, residuals, yerr=yerr, 
                            fmt=marker, color=COLOUR, alpha=alpha,
                            mec=None)
            ax2[1].errorbar(m2, residuals, yerr=yerr, 
                            fmt=marker, color=COLOUR, alpha=alpha,
                            mec=None)
            ax2[2].errorbar(m3, residuals, yerr=yerr, 
                            fmt=marker, color=COLOUR, alpha=alpha,
                            mec=None)
        except:
            pass
    
    if binned:
        # flatten arrays
        for key, values in params_dict.items():
            params_dict[key] = flatten_array(values)
        
        ret12 = binned_statistic_2d(params_dict['m1'], params_dict['m2'], 
                                    params_dict['residuals'], 
                                    statistic='mean', bins = [20, 40])
        ret13 = binned_statistic_2d(params_dict['m1'], params_dict['m3'], 
                                    params_dict['residuals'], 
                                    statistic='mean', bins = [20, 40])
        ret23 = binned_statistic_2d(params_dict['m2'], params_dict['m3'], 
                                    params_dict['residuals'], 
                                    statistic='mean', bins = [40, 40])

        extent12 = get_extent(ret12)
        extent13 = get_extent(ret13)
        extent23 = get_extent(ret23)
        ax[0,0].imshow(ret12[0].T, cmap=CMAP, extent=extent12, norm=norm,
                  interpolation='nearest', origin='lower', aspect='auto')
        ax[1,0].imshow(ret13[0].T, cmap=CMAP, extent=extent13,norm=norm,
                  interpolation='nearest', origin='lower', aspect='auto')
        ax[1,1].imshow(ret23[0].T, cmap=CMAP, extent=extent23, norm=norm, 
                  interpolation='nearest', origin='lower', aspect='auto')
        
    # separated plots
    # bins
    min_val1 = np.round(params_dict['m1'], 0).min()
    max_val1 = np.round(params_dict['m1'], 0).max()
    bins1 = np.arange(min_val1, max_val1+step1, step1)
    delta_bins1, std_bins1, err_bins1 = bin_values(bins1, 
                                                params_dict['m1'], 
                                                params_dict['residuals'], 
                                                params_dict['yerr'])
    
    ax2[0].errorbar(bins1[:-1], delta_bins1, err_bins1, 
                color=BIN_COLOUR, zorder=30, ms=8, mec='k', fmt='h', 
                ecolor=BIN_COLOUR, capsize=8, capthick=2, elinewidth=2.5)
    
    min_val2 = np.round(params_dict['m2'], 0).min()
    max_val2 = np.round(params_dict['m2'], 0).max()
    bins2 = np.arange(min_val2, max_val2+step2, step2)
    delta_bins2, std_bins2, err_bins2 = bin_values(bins2, 
                                                params_dict['m2'], 
                                                params_dict['residuals'], 
                                                params_dict['yerr'])
    
    ax2[1].errorbar(bins2[:-1], delta_bins2, err_bins2, 
                color=BIN_COLOUR, zorder=30, ms=8, mec='k', fmt='h', 
                ecolor=BIN_COLOUR, capsize=8, capthick=2, elinewidth=2.5)
    
    min_val3 = np.round(params_dict['m3'], 0).min()
    max_val3 = np.round(params_dict['m3'], 0).max()
    bins3 = np.arange(min_val3, max_val3+step3, step3)
    delta_bins3, std_bins3, err_bins3 = bin_values(bins3, 
                                                params_dict['m3'], 
                                                params_dict['residuals'], 
                                                params_dict['yerr'])
    
    ax2[2].errorbar(bins3[:-1], delta_bins3, err_bins3, 
                color=BIN_COLOUR, zorder=30, ms=8, mec='k', fmt='h', 
                ecolor=BIN_COLOUR, capsize=8, capthick=2, elinewidth=2.5)
    
    residuals, yerr = params_dict['residuals'], params_dict['yerr']
    m1 = params_dict['m1']
    if mask_epochs is not None:
        xmin, xmax = mask_epochs
    else:
        xmin, xmax = m1.min(), m1.max()
        
    mask = (m1>=xmin) & (m1<=xmax)
    y, w = residuals[mask], 1/yerr[mask]**2   
    
    wmean = np.average(y, weights=w)
    werr = np.sqrt(np.sum(w*(y- wmean)**2)/((len(w)-1)*np.sum(w)))
    wstd = np.sqrt(np.average((y - wmean)**2, weights=w))
    
    ax2[0].hlines(wmean, xmin, xmax, colors=BIN_COLOUR, 
                          linestyles='dashed', lw=3, zorder=20)
    ax2[0].hlines(wmean+wstd, xmin, xmax, colors=BIN_COLOUR, 
                          linestyles='dotted', lw=3, zorder=20)
    ax2[0].hlines(wmean-wstd, xmin, xmax, colors=BIN_COLOUR, 
                          linestyles='dotted', lw=3, zorder=20)
    
    if 'max' in param:
        param = param[0]+'$_{max}$'

    m1_label = 'Phase of the closest\nNIR epoch to T$_{max}$'
    m2_label = 'Mean phase of the\nNIR epochs (days)'
    m3_label = 'Difference between furthest\nNIR epochs (days)'
    title = f'Reference: {ref_bands} ($\Delta=${wmean:.3f}, $\sigma=${wstd:.3f})'
    residual_label = f'$\Delta${param} (mag)\n(Sim$-$Ref)'
    
    ax[0,0].set_xlabel(m1_label, fontsize=16)
    ax[0,0].set_ylabel(m2_label, fontsize=16)
    ax[1,0].set_xlabel(m1_label, fontsize=16)
    ax[1,0].set_ylabel(m3_label, fontsize=16)
    ax[1,1].set_xlabel(m2_label, fontsize=16)
    ax[0,0].set_title(title, fontsize=16)
    ax[1,1].axes.yaxis.set_visible(False)
    
    # finish colour bar
    ax[0,1].set_aspect(6)
    cbar = plt.colorbar(s_m, cax=ax[0, 1])
    cbar.set_label(residual_label, rotation=270, fontsize=16)
    cbar.ax.get_yaxis().labelpad = 25
    cbar.ax.tick_params(labelsize=16)
    
    for axes in ax:
        for subaxis in axes:
            subaxis.tick_params(labelsize=16)
            
    ax2[0].axhline(0, color='k', ls='--', zorder=20)
    ax2[0].set_xlabel(m1_label.replace('\n', ''), fontsize=16)
    ax2[0].set_ylabel(residual_label, fontsize=16)
    ax2[0].set_title(title, fontsize=16)
    ax2[0].tick_params(labelsize=16)
    ax2[0].set_ylim(-0.15, 0.15)
    
    ax2[1].axhline(0, color='k', ls='--', zorder=20)
    ax2[1].set_xlabel(m2_label.replace('\n', ''), fontsize=16)
    ax2[1].set_ylabel(residual_label, fontsize=16)
    ax2[1].tick_params(labelsize=16)
    ax2[1].set_ylim(-0.15, 0.15)
    
    ax2[2].axhline(0, color='k', ls='--', zorder=20)
    ax2[2].set_xlabel(m3_label.replace('\n', ''), fontsize=16)
    ax2[2].set_ylabel(residual_label, fontsize=16)
    ax2[2].tick_params(labelsize=16)
    ax2[2].set_ylim(-0.15, 0.15)
    
    fig.tight_layout()
    fig2.tight_layout()
    plt.show()
    
def master_plot(param, ref_bands, nir_bands, n_epochs, binned=True, 
                        step1=4, step2=10, step3=10, mask_epochs=None):
    """Plots any of the diagnosis plots.
    
    Parameters
    ==========
    params: str
        SN parameter (e.g., Jmax, Tmax, st).
    ref_bands: str
        Bands used for the reference fits. E.g., `AllBands`,
        `Allminusu`, 'grJH', 'gr' and 'gp' (gaussian process).
    nir_bands: str
        NIR bands used for the simulations fits. E.g., `J`, 
        `H` or `JH`.
    n_epochs: int
        Number of epochs for the combinatories of the 
        simulations.
    binned: bool, default `True`
        If `True`, plots the 2D binned version when 
        `n_epochs` is 2 or 3.
    mask_epochs: list-like or tuple, default `None`
        Range in days where to mask metric 1 to calculate
        the global offset and scatter. E.g., (-8, 10).
    """
    # assertions
    param_options = ['Tmax', 'st', 'Jmax', 'Hmax', 'gmax', 'rmax']
    param_message = ('`param` must be one of the following:' 
                   f'{param_options}')
    assert param in param_options, param_message
    
    ref_bands_options = ['AllBands', 'Allminusu', 
                         'grJH', 'gr', 'gp']
    ref_message = ('`ref_bands` must be one of the following:' 
                   f'{ref_bands_options}')
    assert ref_bands in ref_bands_options, ref_message
    
    nir_bands_options = ['J', 'H', 'JH']
    nir_message = ('`nir_bands` must be one of the following:' 
                   f'{nir_bands_options}')
    assert nir_bands in nir_bands_options, nir_message
    
    if 'J' in param:
        message = ('Jmax was not fitted with this configuration:'
                 f'`nir_bands`=`{nir_bands}`')
        assert 'J' in nir_bands, message
    if 'H' in param:
        message = ('Hmax was not fitted with this configuration:'
                 f'`nir_bands`=`{nir_bands}`')
        assert 'H' in nir_bands, message
    
    assert 1<=n_epochs<=3, '`n_epochs` must be 1, 2 or 3'
    
    # plots
    if n_epochs==1:
        plot_m1(param, ref_bands, nir_bands, step1, mask_epochs)
    elif n_epochs==2:
        plot_m2(param, ref_bands, nir_bands, binned, 
                                    step1, step2, mask_epochs)
    elif n_epochs==3:
        plot_m3(param, ref_bands, nir_bands, binned, 
                                    step1, step2, step3, mask_epochs)

In [ ]:
parameter = 'Jmax'  # Jmax, Hmax, gmax, rmax, Tmax
ref_bands = 'AllBands'  # AllBands, Allminus, grJH, gr, gp (gaussian process)
nir_bands = 'J'  # J, H, JH
n_epochs = 3  # 1, 2, 3
mask_epochs= (-5, 10)  # None, (-10, 10)

master_plot(parameter, ref_bands, nir_bands, 
                n_epochs, mask_epochs=mask_epochs)